In [ ]:
%pip install transformers Pillow

In [ ]:
target_directory = '/notebooks/lora_training/Blockout/blockout'
text = "a basemesh blockout of"

In [ ]:
import os
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

# Load the model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

In [ ]:
def process_image(img_path):
    try:
        # Load and process the image using Pillow
        raw_image = Image.open(img_path).convert('RGB')
    except IOError:
        print(f'Could not read image {img_path}. Skipping.')
        return None
    
    # Process the image and generate a caption
    inputs = processor(raw_image, text, return_tensors="pt").to("cuda")
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

def save_caption(caption, img_path):
    # Create a text file with the same name as the image
    txt_path = os.path.splitext(img_path)[0] + '.txt'
    with open(txt_path, 'w') as file:
        file.write(caption)

# Walk through the target directory including subdirectories
for root, dirs, files in os.walk(target_directory):
    for file in files:
        # Check if the file is an image (by extension)
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.webp')):
            img_path = os.path.join(root, file)
            caption = process_image(img_path)
            if caption is not None:
                save_caption(caption, img_path)
                print(f'Processed {img_path}')